In [ ]:
# Реализуем процесс обучения адаптера

# Загрузить модель. 
# Загрузить лору. Какую модель будем использовать?
# Загрузить данные для дообучения модели.

# Откуда взять датасет для дообучения модели.
# 1. Из урока. В ноутбуке есть ссылка на датасет для дообучение мод ели.
# 2. Из датасет а Ильи Гусева. Попробуем взять его.

# Надо посмотреть как можно загружать на диск данные с google-colab.

In [5]:
import os
import sys
import json
# import torch
import pandas as pd
from tqdm import tqdm
from typing import List
# from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
# from peft import PeftConfig, PeftModel
# import peft

In [20]:
df = pd.read_csv("all_recepies_inter.csv", sep="\t", usecols=["name", "Инструкции"]).rename(columns={"Инструкции": "instructions"})

In [21]:
df.head()

,name,instructions
0,рассольник классический с перловкой и солеными...,Подготовить указанные ингредиенты для приготов...
1,Суп пюре из белокочаной капусты,"Необходимые ингредиенты\r\nНарезаем лук, морко..."
2,Постные щи из квашеной капусты,"Честно признаюсь, у меня не было репы на момен..."
3,Тюря- простой суп быстро и вкусно,"\r\nНачинаем мы приготовление тюри с того, что..."
4,Фасолевый суп из красной фасоли,Подготовить ингредиенты. Для приготовления суп...
...,...,...
27879,Салат с филе тунца и огурцами в азиатской…,1. Для приготовления азиатского соуса смешать ...
27880,Теплый салат с куриной печенью и яблоками,"1. Маленькими ломтиками порезать яблоки, запек..."
27881,Самый зеленый салат,"1. Чеснок натереть, сыр фета раскрошить.\r\n2...."
27882,Теплый салат с тыквой и брынзой под…,"1. Тыкву нарезаем кубиками, добавляем оливково..."


## Формируем датасет для обучения модели.

In [43]:
dialoges = []
for i in tqdm(range(len(df))):
    dialog_format = {
    "messages": [
    {"role": "user", "content": df.iloc[i]['name']},
    {"role": "bot", "content": df.iloc[i]['instructions']}
    ],
     "source": "alpaca"
    }
    # dialog_format['messages'][0]['content'] = 
    # dialog_format['messages'][1]['content'] = 
    dialoges.append(dialog_format)

100%|██████████| 27884/27884 [00:01<00:00, 16014.98it/s]


In [45]:
tran_dataset_dir = 'datasets'
recepties= 'recepties.jsonl'

In [46]:
os.makedirs(tran_dataset_dir, exist_ok=True)

In [47]:
with open(os.path.join(tran_dataset_dir, recepties), "w", encoding="utf-8") as w:
    for d in tqdm(dialoges):
        w.write(json.dumps(d, ensure_ascii=False) + "\n")

100%|██████████| 27884/27884 [00:00<00:00, 59712.41it/s]


In [ ]:
# Сохраненение датасета в файл.

In [ ]:
# Скрипты для обучения адаптера.
# Во время обучения будем периодически останавливать обучение модели и проверять качество.
# Посмотреть как проходит обучение модели на диалогах.
# Необходимо дообучить существующий адаптер.

In [ ]:
import subprocess
from pathlib import Path

# Set up paths
content_dir = Path('.').resolve()
train_full_path = content_dir / 'train_full.jsonl'
val_full_path = content_dir / 'val_full.jsonl'

# Run create_chat_set script from rulm
module_directory = Path('rulm/self_instruct').resolve()
subprocess.run(
    ['python', '-m', 'src.data_processing.create_chat_set', str(train_full_path), str(val_full_path)],
    cwd=module_directory,
    check=True
)

# Check if train_full.jsonl exists
if not train_full_path.exists():
    raise FileNotFoundError(f"{train_full_path} does not exist")